# Desafio DataSprints - Fabio Kfouri

Este é um desafio dado pela <b><i>data <span style='color: red'>sprints</span></i></b> para avaliação técnica em Engenharia de Dados.

In [1]:
try:
    !pip install pyspark=="2.4.5" --quiet
    !pip install pandas=="1.0.4" --quiet
    !pip install boto3=="1.13.1" --quiet
except:
    print("Running throw py file.")

In [2]:
from pyspark import SparkContext as sc
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pyspark
import pandas as pd
import json

Criando uma SparkSession

In [3]:
spark = SparkSession\
        .builder\
        .appName("Desafio Data Sprints - Fabio Kfouri")\
        .getOrCreate()
spark

## Criação dos Dataframes
Referenciando o endereço das fontes. Para desenvolvimento local foi incluido as fontes na pasta <b>/data/</b>.

Para uso em cluster, sera usado o endereço do Bucket AWS <b>s3://data-sprints-eng-test/</b>.

In [4]:
import os
dirpath = os.getcwd()

dataPath = 'https://s3.amazonaws.com/data-sprints-eng-test/'
dataPath = 's3://data-sprints-eng-test/'
dataPath = dirpath + "/data/"
dataPath

'E:\\Projetos\\Jobs\\DataSprints/data/'

#### Leitura e Correçao da fonte Payment

In [5]:
df_payment = spark.read.csv(dataPath + 'data-payment_lookup-csv.csv', header = True, sep = ",")
df_payment.show(3)

+------------+--------------+
|           A|             B|
+------------+--------------+
|payment_type|payment_lookup|
|         Cas|          Cash|
|         CAS|          Cash|
+------------+--------------+
only showing top 3 rows



Verificado que a primeira linha precisa ser ignorada. Inclusao de index para auxiliar a correção.

In [6]:
df_payment = df_payment.withColumn('index', F.monotonically_increasing_id())
df_payment.show(3)

+------------+--------------+-----+
|           A|             B|index|
+------------+--------------+-----+
|payment_type|payment_lookup|    0|
|         Cas|          Cash|    1|
|         CAS|          Cash|    2|
+------------+--------------+-----+
only showing top 3 rows



Renomeando a coluna pelos valores pelo index zero. 

In [7]:
cols = df_payment.columns
values = df_payment.filter('index = 0').collect()
#print(cols, len(cols), values, len(values[0]), values[0])

for i in range(len(cols)):
    if cols[i] != 'index' :
        #print(i, cols[i], values[0][i])
        df_payment = df_payment.select(df_payment.columns).withColumnRenamed(cols[i], values[0][i])

Removendo a linha index zero.

In [8]:
df_payment = df_payment.filter('index<>0')
df_payment.show(3)

+------------+--------------+-----+
|payment_type|payment_lookup|index|
+------------+--------------+-----+
|         Cas|          Cash|    1|
|         CAS|          Cash|    2|
|         Cre|        Credit|    3|
+------------+--------------+-----+
only showing top 3 rows



Criação de view payment

In [10]:
df_payment.createOrReplaceTempView("payment")

#### Leitura da fonte de Vendor

In [9]:
df_vendor = spark.read.csv(dataPath + 'data-vendor_lookup-csv.csv', header = True, sep = ",")
df_vendor.show()

+---------+--------------------+--------------------+--------+-----+-----+-------+--------------------+-------+
|vendor_id|                name|             address|    city|state|  zip|country|             contact|current|
+---------+--------------------+--------------------+--------+-----+-----+-------+--------------------+-------+
|      CMT|Creative Mobile T...|950 4th Road Suit...|Brooklyn|   NY|11210|    USA|contactCMT@gmail.com|    Yes|
|      VTS|        VeriFone Inc|       26 Summit St.|Flushing|   NY|11354|    USA|   admin@vtstaxi.com|    Yes|
|      DDS|Dependable Driver...|8554 North Homest...|   Bronx|   NY|10472|    USA|          9778896500|    Yes|
|       TS|  Total Solutions Co|Five Boroughs Tax...|Brooklyn|   NY|11229|    USA|       mgmt@5btc.com|    Yes|
|       MT|           Mega Taxi| 4 East Jennings St.|Brooklyn|   NY|11228|    USA|contact@megataxic...|     No|
+---------+--------------------+--------------------+--------+-----+-----+-------+--------------------+-

Criação da view vendor.

In [11]:
df_vendor.createOrReplaceTempView("vendor")

## Leitura das corridas de taxi no período de 2009 à 2012

In [12]:
f_2009 = 'data-sample_data-nyctaxi-trips-2009-json_corrigido.json'
df_2009 = spark.read.json(dataPath + f_2009)
df_2009.count()

1000000

In [13]:
f_2010 = 'data-sample_data-nyctaxi-trips-2010-json_corrigido.json'
df_2010 = spark.read.json(dataPath + f_2010)

In [14]:
f_2011 = 'data-sample_data-nyctaxi-trips-2011-json_corrigido.json'
df_2011 = spark.read.json(dataPath + f_2011)

In [15]:
f_2012 = 'data-sample_data-nyctaxi-trips-2012-json_corrigido.json'
df_2012 = spark.read.json(dataPath + f_2012)

## Preparação do DataFrame de corridas de taxi.
Concatenando todos os dataFrames em único DataFrame e em seguinda verificando o total de linhas do DataFrame.

In [16]:
df = df_2012.union(df_2011).union(df_2010).union(df_2009)
print("Tamanho do DataFrame concatenado:", df.count(), 'registros')

Tamanho do DataFrame concatenado: 4000000 registros


Identificando o Schema do DataFrame

In [17]:
df.printSchema()

root
 |-- dropoff_datetime: string (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: long (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- vendor_id: string (nullable = true)



Visualizando o aspecto dos dados

In [18]:
df.show(5, truncate = False)

+--------------------------------+----------------+-----------------+-----------+---------------+------------+--------------------------------+---------------+----------------+---------+------------------+---------+----------+------------+------------+-------------+---------+
|dropoff_datetime                |dropoff_latitude|dropoff_longitude|fare_amount|passenger_count|payment_type|pickup_datetime                 |pickup_latitude|pickup_longitude|rate_code|store_and_fwd_flag|surcharge|tip_amount|tolls_amount|total_amount|trip_distance|vendor_id|
+--------------------------------+----------------+-----------------+-----------+---------------+------------+--------------------------------+---------------+----------------+---------+------------------+---------+----------+------------+------------+-------------+---------+
|2012-06-09T16:31:19.862787+00:00|40.74795        |-73.994712       |5.4        |2              |Cash        |2012-06-09T16:21:13.580003+00:00|40.74295       |-74.004114

Conversão de colunas [dropoff_datetime, pickup_datetime] do tipo String para tipo TimeStamp.

In [19]:
#DataFrame Convertido (dfc)
dfc = df.withColumn('dropoff_datetime', F.to_utc_timestamp('dropoff_datetime', "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"))\
        .withColumn('pickup_datetime', F.to_utc_timestamp('dropoff_datetime', "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"))
dfc.printSchema()

root
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: long (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- vendor_id: string (nullable = true)



Visualizando o aspecto dos dados, em especial os campos dataTime

In [20]:
dfc.show(5, False)

+--------------------------+----------------+-----------------+-----------+---------------+------------+--------------------------+---------------+----------------+---------+------------------+---------+----------+------------+------------+-------------+---------+
|dropoff_datetime          |dropoff_latitude|dropoff_longitude|fare_amount|passenger_count|payment_type|pickup_datetime           |pickup_latitude|pickup_longitude|rate_code|store_and_fwd_flag|surcharge|tip_amount|tolls_amount|total_amount|trip_distance|vendor_id|
+--------------------------+----------------+-----------------+-----------+---------------+------------+--------------------------+---------------+----------------+---------+------------------+---------+----------+------------+------------+-------------+---------+
|2012-06-09 13:31:19.862787|40.74795        |-73.994712       |5.4        |2              |Cash        |2012-06-09 13:31:19.862787|40.74295       |-74.004114      |null     |null              |0.0      |0.

Criando uma view trip.

In [111]:
dfc.createOrReplaceTempView("trip")

## Questões do Quesito Mínimo

<b>O resultado para a questão 1 sobre a distância média (valor arredondado) percorrida por viagens com no máximo 2 passareios é:

In [113]:
df_question_1 = spark.sql("""
      SELECT ROUND(AVG(trip_distance),3) mean_trip_distance
        FROM trip t
       WHERE t.passenger_count <= 2
""")
df_question_1.show()

+------------------+
|mean_trip_distance|
+------------------+
|             2.663|
+------------------+



O resultado para a questão 2 referente aos 3 maiores Vendors em quantidade de dinheiro (valores em Milhões U$) arrecado são:

In [116]:
df_question_2 = spark.sql("""
    SELECT v.name, t.vendor_id, ROUND(SUM(total_amount)/1E6,3) amount 
      FROM trip t LEFT JOIN vendor v ON (t.vendor_id = v.vendor_id)
  GROUP BY t.vendor_id, v.name
  ORDER BY SUM(total_amount) DESC
     LIMIT 3
""")
df_question_2.show(truncate = False)

+---------------------------------+---------+------+
|name                             |vendor_id|amount|
+---------------------------------+---------+------+
|Creative Mobile Technologies, LLC|CMT      |19.549|
|VeriFone Inc                     |VTS      |19.043|
|Dependable Driver Service, Inc   |DDS      |2.715 |
+---------------------------------+---------+------+



In [94]:
df_question_2 = spark.sql("""
    SELECT payment_type, fare_amount, surcharge, tip_amount, tolls_amount, total_amount, t.vendor_id, v.name
      FROM trip t JOIN vendor v ON (t.vendor_id = v.vendor_id)
""")

In [95]:
df_question_2.show()

+------------+-----------+---------+----------+------------+------------+---------+--------------------+
|payment_type|fare_amount|surcharge|tip_amount|tolls_amount|total_amount|vendor_id|                name|
+------------+-----------+---------+----------+------------+------------+---------+--------------------+
|        Cash|        5.4|      0.0|       0.0|         0.0|         5.4|      CMT|Creative Mobile T...|
|        Cash|       15.4|      0.0|       0.0|         0.0|        15.4|      CMT|Creative Mobile T...|
|        CASH|        6.1|      1.0|       0.0|         0.0|         7.1|      VTS|        VeriFone Inc|
|        CASH|       12.9|      0.5|       0.0|         0.0|        13.4|      VTS|        VeriFone Inc|
|        CASH|        7.7|      0.5|       0.0|         0.0|         8.2|      VTS|        VeriFone Inc|
|        CASH|        6.5|      0.0|       0.0|         0.0|         6.5|      VTS|        VeriFone Inc|
|        CASH|       22.1|      0.0|       0.0|        

+---------------------------------+---------+------+
|name                             |vendor_id|amount|
+---------------------------------+---------+------+
|Creative Mobile Technologies, LLC|CMT      |19.549|
|VeriFone Inc                     |VTS      |19.043|
|Dependable Driver Service, Inc   |DDS      |2.715 |
+---------------------------------+---------+------+



In [85]:
df_question_2.show()

+--------------------+-------+---------+--------------------+
|              amount|    qtd|vendor_id|                name|
+--------------------+-------+---------+--------------------+
|1.9549084279999983E7|1916128|      CMT|Creative Mobile T...|
|1.9043434000000075E7|1833052|      VTS|        VeriFone Inc|
|  2714901.7200000007| 250816|      DDS|Dependable Driver...|
|                45.6|      4|       TS|  Total Solutions Co|
+--------------------+-------+---------+--------------------+



In [ ]:
aa